In [1]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist, squareform, pdist
from scipy.stats import ks_2samp
from scipy.stats import pearsonr



import os,csv,re
import math
from skimage import io, color

from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.model_selection import train_test_split
from anndata import AnnData, read_h5ad
import seaborn as sns

import json

from spaotsc import SpaOTsc

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_merfish = read_h5ad("data/liver_merfish.h5ad")

In [3]:
Rdata_ind, Qdata_ind, _, _ =train_test_split(range(data_merfish.shape[0]), data_merfish.obs['louvain'], test_size=0.5,random_state=1,stratify=data_merfish.obs['louvain'])

Rdata = data_merfish[np.sort(Rdata_ind), :]
Qdata = data_merfish[np.sort(Qdata_ind), :]

## Cannot run on the entire dataset, downsampling is needed
Rdata_ind, Qdata_ind, _, _ =train_test_split(range(Rdata.shape[0]), Rdata.obs['louvain'], test_size=0.25,random_state=1,stratify=Rdata.obs['louvain'])
Rdata = Rdata[np.sort(Qdata_ind), :]

Rdata_ind, Qdata_ind, _, _ =train_test_split(range(Qdata.shape[0]), Qdata.obs['louvain'], test_size=0.25,random_state=1,stratify=Qdata.obs['louvain'])
Qdata = Qdata[np.sort(Qdata_ind), :]

print((Rdata.shape, Qdata.shape))

((78222, 500), (78223, 500))


In [5]:
datatest = Qdata.copy()
datatrain = Rdata.copy()
random.seed(2021)
torch.manual_seed(2021)
np.random.seed(2021)

## Running spaOTsc
df_sc = pd.DataFrame(datatest.X)
is_dmat = cdist(np.array(datatrain.obs.iloc[:,0:2]), np.array(datatrain.obs.iloc[:,0:2]), 'euclidean')
sc_dmat = cdist(datatest.X, datatest.X, 'euclidean')

In [6]:
spsc = SpaOTsc.spatial_sc(sc_data=df_sc, is_dmat=is_dmat, sc_dmat=sc_dmat)
cost_matrix = cdist(datatest.X, datatrain.X, 'euclidean')
        
location_pred = spsc.transport_plan(cost_matrix)

In [9]:
location_sum = np.sum(location_pred, axis=1)
location_pred_copy = location_pred / location_sum.reshape(len(location_sum), 1)

pred_cord_transform = location_pred_copy.dot(np.array(Rdata.obs[['x_cord', 'y_cord']]))
np.save("output/liver/spaotsc_liver.npy", pred_cord_transform)